# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [61]:
import torch as th

In [62]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [63]:
y = x + x

In [64]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [65]:
import syft as sy

In [66]:
hook = sy.TorchHook(th)

W0705 16:02:44.548796 11660 hook.py:97] Torch was already hooked... skipping hooking process


In [67]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [68]:
bob = sy.VirtualWorker(hook, id="bob")

In [69]:
bob._objects

{}

In [70]:
x = th.tensor([1,2,3,4,5])

In [71]:
x = x.send(bob)

In [72]:
bob._objects

{52559349317: tensor([1, 2, 3, 4, 5])}

In [73]:
x.location

<VirtualWorker id:bob #objects:1>

In [74]:
x.id_at_location

52559349317

In [75]:
x.id

63499809022

In [76]:
x.owner

<VirtualWorker id:me #objects:0>

In [77]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [78]:
x

(Wrapper)>[PointerTensor | me:63499809022 -> bob:52559349317]

In [79]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [80]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [81]:
# try this project here!

In [82]:
alice = sy.VirtualWorker(hook, id = "alice")

In [83]:
def send_tensor_to_workers(tensor, workers =  []):
    y = tensor.send(*workers)
    return  y

def get_tensor(tensor):
    y = tensor.get()
    return y
    

In [84]:
y = th.Tensor([10,2,3,4])
workers = [bob,alice]
y = send_tensor_to_workers(y, workers =workers )

for worker in workers:
    print("Objects of worker {} :{} ".format(worker.id, worker._objects))

y = get_tensor(y)

print("")
print("Tensor:{}".format(y))

Objects of worker bob :{16403660884: tensor([10.,  2.,  3.,  4.])} 
Objects of worker alice :{86618182561: tensor([10.,  2.,  3.,  4.])} 

Tensor:[tensor([10.,  2.,  3.,  4.]), tensor([10.,  2.,  3.,  4.])]


# Lesson: Introducing Remote Arithmetic

In [85]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [86]:
x

(Wrapper)>[PointerTensor | me:564890580 -> bob:88034512379]

In [87]:
y

(Wrapper)>[PointerTensor | me:57169692518 -> bob:92059012416]

In [88]:
z = x + y

In [89]:
z

(Wrapper)>[PointerTensor | me:62261077189 -> bob:52033151247]

In [90]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [91]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:64254112051 -> bob:29376037834]

In [92]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [93]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [94]:
z = (x + y).sum()


In [95]:
z.backward()

(Wrapper)>[PointerTensor | me:40925180237 -> bob:14769626311]

In [96]:
x = x.get()

In [97]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [36]:
x.grad

NameError: name 'x' is not defined

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [42]:
# try this project here!

In [43]:
bob = sy.VirtualWorker(hook, id = "bob")

In [44]:
import torch as th
from torch.autograd import Variable
X =th.tensor([[1.0,1.0],[2.0,1.0],[3.0,1.0],[4.0,1.0],[5.0,1.0],[6.0,1.0]]
             ,requires_grad = True )


y = th.tensor([[2.0],[4.0],[6.0],[8.0],[10.0],[12.0]]
              ,requires_grad = True)



(X*y).shape

torch.Size([6, 2])

### Send Tensors to Bob

In [45]:
X = X.send(bob)
y = y.send(bob)

## Model Definition

In [46]:
class LinearRegression():
    
    def __init__(self):
        
        #Can initialize weigths with zeros, not on neural nets.
        self.w = th.tensor( [[0.0],[0.0]], requires_grad = True) 
        
        #send to bob
        self.w =  self.w.send(bob)
        
        
            
    def forward(self,x):
        y = th.matmul(x,self.w)
        
        return y
    
    def backward(self, loss):
        """
        loss: torch Tensor
        recieves outouput of linear model
        return gradient with respect to weights
        """
     
        loss.backward()
        
        
    def gradient_step(self,learning_rate):
        
        assert self.w.grad  is not None,"Compute gradients first."
        
        self.w.data.sub_(learning_rate * self.w.grad)
        
        self.w.grad *= 0
        

        
    def l2_norm(self,y,y_pred):
        """
        y:torch tensor
            targets
        y_pred:torch tensor
            predictions make by model
        compute and return l-2 norm between y and y_pred:
        """
        
        m = y.shape[0]
        diff = y - y_pred
        
        
        return th.matmul(diff.t(), diff)/m

In [47]:
model = LinearRegression()

In [48]:


for _ in range(25):
    
    
    y_pred = model.forward(X)
    
    #print("y_pred",y_pred.clone().get())
    

    #Compute cost function
    loss = model.l2_norm(y, y_pred)
    print("Loss", loss.clone().get().data.item())
   
    #Compute gradients of loss with respect to parameters w
    model.backward(loss)


    #print("Gradients:")
    #print("for w:")

    #w_grad = model.w.grad.clone().get()
    #print(w_grad)

    #print("for b:")
    #b_grad = model.b.grad.clone().get()
    #print(b_grad)
    
    
    #Gradient descent step
    model.gradient_step(learning_rate = 0.05)
    
       
    
#.get() params causes to left bob. Use .clone() to make a copy
print("Parameters w: {}".format(model.w.clone().get()))


Loss 60.66666793823242
Loss 21.747970581054688
Loss 7.810120105743408
Loss 2.8180999755859375
Loss 1.029659390449524
Loss 0.38846683502197266
Loss 0.15813873708248138
Loss 0.07496897131204605
Loss 0.04452216997742653
Loss 0.032979678362607956
Loss 0.028230242431163788
Loss 0.025935737416148186
Loss 0.024541640654206276
Loss 0.02349088154733181
Loss 0.022582896053791046
Loss 0.02174530178308487
Loss 0.020951509475708008
Loss 0.020191175863146782
Loss 0.019460072740912437
Loss 0.01875598169863224
Loss 0.018077662214636803
Loss 0.01742393523454666
Loss 0.01679382286965847
Loss 0.0161865446716547
Loss 0.015601209364831448
Parameters w: tensor([[1.9347],
        [0.2795]], requires_grad=True)


In [49]:
x_ =th.tensor([[3.0,1.0],[5.0,1.0],[20.0,1.0]])
print(x_.shape)
x_ = x_.send(bob)

y = model.forward( x_  )
y.clone().get()

torch.Size([3, 2])


tensor([[ 6.0837],
        [ 9.9531],
        [38.9737]], requires_grad=True)

# Lesson: Garbage Collection and Common Errors


In [50]:
bob = bob.clear_objects()

In [51]:
bob._objects

{}

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{8316222502: tensor([1, 2, 3, 4, 5])}

In [54]:
del x

In [55]:
bob._objects

{}

In [56]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [57]:
bob._objects

{68024529100: tensor([1, 2, 3, 4, 5])}

In [58]:
x = "asdf"

In [59]:
bob._objects

{}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
x

(Wrapper)>[PointerTensor | me:21709422336 -> bob:19551685457]

In [62]:
bob._objects

{19551685457: tensor([1, 2, 3, 4, 5])}

In [63]:
x = "asdf"

In [64]:
bob._objects

{19551685457: tensor([1, 2, 3, 4, 5])}

In [65]:
del x

In [66]:
bob._objects

{19551685457: tensor([1, 2, 3, 4, 5])}

In [67]:
bob = bob.clear_objects()
bob._objects

{}

In [68]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [69]:
bob._objects

{79860924165: tensor([1, 2, 3, 4, 5])}

In [70]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [71]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:49591635336 -> bob:71934871717]
Tensor B: tensor([1, 1, 1, 1, 1])

In [72]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

NameError: name 'alice' is not defined

In [73]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:2493730555 -> bob:82364473725]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [81]:
from torch import nn, optim

In [82]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [83]:
# A Toy Model
model = nn.Linear(2,1)

In [84]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [85]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(8.2979)
tensor(0.8427)
tensor(0.2249)
tensor(0.1372)
tensor(0.1002)
tensor(0.0750)
tensor(0.0564)
tensor(0.0425)
tensor(0.0321)
tensor(0.0243)
tensor(0.0184)
tensor(0.0140)
tensor(0.0106)
tensor(0.0081)
tensor(0.0062)
tensor(0.0047)
tensor(0.0036)
tensor(0.0027)
tensor(0.0021)
tensor(0.0016)


In [86]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [87]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

NameError: name 'alice' is not defined

In [88]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

NameError: name 'data_alice' is not defined

In [89]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    print("Model", model.parameters())
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [90]:
train()

Model <generator object Module.parameters at 0x000001A09D8AA308>
tensor(60.3283, requires_grad=True)
tensor(4995.4478, requires_grad=True)
tensor(39824312., requires_grad=True)
tensor(7.0661e+09, requires_grad=True)
tensor(6.8421e+11, requires_grad=True)
tensor(5.4653e+15, requires_grad=True)
tensor(9.6972e+17, requires_grad=True)
tensor(9.3898e+19, requires_grad=True)
tensor(7.5004e+23, requires_grad=True)
tensor(1.3308e+26, requires_grad=True)
tensor(1.2886e+28, requires_grad=True)
tensor(1.0293e+32, requires_grad=True)
tensor(1.8263e+34, requires_grad=True)
tensor(1.7684e+36, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [91]:
bob.clear_objects()
alice.clear_objects()

NameError: name 'alice' is not defined

In [92]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [93]:
x = x.send(alice)

NameError: name 'alice' is not defined

In [94]:
bob._objects

{75396253104: tensor([1, 2, 3, 4, 5])}

In [95]:
alice._objects

NameError: name 'alice' is not defined

In [96]:
y = x + x

In [97]:
y

(Wrapper)>[PointerTensor | me:14708506536 -> bob:38466281331]

In [98]:
bob._objects

{75396253104: tensor([1, 2, 3, 4, 5]),
 38466281331: tensor([ 2,  4,  6,  8, 10])}

In [99]:
alice._objects

NameError: name 'alice' is not defined

In [100]:
jon = sy.VirtualWorker(hook, id="jon")

In [101]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

NameError: name 'alice' is not defined

In [102]:
bob._objects

{}

In [103]:
alice._objects

NameError: name 'alice' is not defined

In [104]:
x = x.get()
x

KeyError: 'Object "75396253104" not found on worker!!!You just tried to interact with an object ID:75396253104 on <VirtualWorker id:bob #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [105]:
bob._objects

{}

In [106]:
alice._objects

NameError: name 'alice' is not defined

In [107]:
x = x.get()
x

KeyError: 'Object "75396253104" not found on worker!!!You just tried to interact with an object ID:75396253104 on <VirtualWorker id:bob #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [108]:
bob._objects

{}

In [109]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

NameError: name 'alice' is not defined

In [110]:
bob._objects

{}

In [111]:
alice._objects

NameError: name 'alice' is not defined

In [112]:
del x

In [113]:
bob._objects

{}

In [114]:
alice._objects

NameError: name 'alice' is not defined

# Lesson: Pointer Chain Operations

In [115]:
bob.clear_objects()
alice.clear_objects()

NameError: name 'alice' is not defined

In [116]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [117]:
bob._objects

{18217518208: tensor([1, 2, 3, 4, 5])}

In [118]:
alice._objects

NameError: name 'alice' is not defined

In [119]:
x.move(alice)

NameError: name 'alice' is not defined

In [120]:
bob._objects

{18217518208: tensor([1, 2, 3, 4, 5])}

In [121]:
alice._objects

NameError: name 'alice' is not defined

In [122]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

NameError: name 'alice' is not defined

In [123]:
bob._objects

{18217518208: tensor([1, 2, 3, 4, 5])}

In [124]:
alice._objects

NameError: name 'alice' is not defined

In [125]:
x.remote_get()

InvalidTensorForRemoteGet: Tensor does not have attribute child. You remote get should be called on a chain of pointer tensors, instead you called it on tensor([1, 2, 3, 4, 5]).

In [126]:
bob._objects

{18217518208: tensor([1, 2, 3, 4, 5])}

In [127]:
alice._objects

NameError: name 'alice' is not defined

In [128]:
x.move(bob)

KeyError: 'Object "90929631441" not found on worker!!!You just tried to interact with an object ID:90929631441 on <VirtualWorker id:bob #objects:1> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [129]:
x

(Wrapper)>[PointerTensor | me:90929631441 -> bob:18217518208]

In [130]:
bob._objects

{18217518208: (Wrapper)>tensor([1, 2, 3, 4, 5])}

In [131]:
alice._objects

NameError: name 'alice' is not defined

# Final Project

- Central server is not trusted.
- One worker will aggregate gradients (take averge).
- Then, send model to central server.

In [155]:
import syft as sy
import torch as th 


In [156]:
hook = sy.TorchHook(th)


W0707 14:45:36.041783 15104 hook.py:97] Torch was already hooked... skipping hooking process


In [157]:
from torch import  nn,optim

In [158]:
# A Toy Dataset
data = th.tensor([[1.0,1.0],[3,1.0],[1,1.0],[0,1.0],[23.0,1.0],[6,1.0],[7,1.0],[15,1.0],[9.0,1.0]], requires_grad=True)


target = th.tensor([[2.],[6.0], [2.0], [0],[46.0],[12.0],[14.0],[30.0],[18.0]], requires_grad=True)
        


In [159]:
m = 3 #number of workers

In [160]:
workers  =[sy.VirtualWorker(hook, id = "w"+ str(i)) for i in range(m)]
chunk_size = data.shape[0]//m



## Make datasets

In [189]:

#make a mini-dataset, one per worker
datasets = [ [
              data[j*chunk_size : (j+1) * chunk_size  ] ,
              target[j*chunk_size : (j+1) * chunk_size] 
             ]
            
             for  j in range(m)
           ]

print("Datasets[0]", *datasets[0], sep= "\n")

#send datasets to workers
for k in range(m):
    
    datasets[k][0] = datasets[k][0].send(workers[k])
    
    datasets[k][1] = datasets[k][1].send(workers[k])
    
    
       

Datasets[0]
tensor([[1., 1.],
        [3., 1.],
        [1., 1.]], grad_fn=<SliceBackward>)
tensor([[2.],
        [6.],
        [2.]], grad_fn=<SliceBackward>)


In [197]:
import random
from random import shuffle

def train_secure_grads(m,datasets,workers, iterations=20):
    """
    iterations: int
    
    m: int
        number of models/workers
        
    datasets: list of lists of tensors
        [ [data_1,targets_1],[data_2, targets_2], ...     ]
    
    workers: list of VirtualWorkers
    
    iterations: int 
        number of SGD steps to do.
        
    """
    # Global model
    global_model = nn.Linear(2,1)
    
    # Create optimizator
    opt = optim.SGD(params = global_model.parameters(), lr = 0.001)
    

    for i,t in enumerate(datasets):
        _data,_target  = t[0],t[1]
        
        if i == 0:
            # Send model to the data
            global_model = global_model.send(_data.location)
            #we a pointer.
        else:
            # Previously we send data to worker 1. global model is a pointer.
            # Move actual data (tensor, not pointer) to next worker.
            global_model.move(_data.location)
        
        
        for _ in range(iterations):            
            # do normal training
            opt.zero_grad()
            pred = global_model(_data)
            
            
            loss = ((pred - _target)**2).sum()
            loss.backward()
            
            opt.step()
            
            print("Loss: {} for worker: {}".format( loss.clone().get().item(), i ))
            
        
       
        
        print("Params {} after training model: {}".format(global_model.weight.clone().get() ,i))
        print()
    
   
    # Get back trained model
    global_model = global_model.get()
        
    print("Params for model global model: {}".format(global_model.weight.clone()))

        
    return global_model


In [198]:
train_secure_grads(m = 3, datasets = datasets,workers = workers, iterations = 50  )

Loss: 33.879798889160156 for worker: 0
Loss: 31.792123794555664 for worker: 0
Loss: 29.835739135742188 for worker: 0
Loss: 28.00237274169922 for worker: 0
Loss: 26.284286499023438 for worker: 0
Loss: 24.674219131469727 for worker: 0
Loss: 23.165363311767578 for worker: 0
Loss: 21.751354217529297 for worker: 0
Loss: 20.426210403442383 for worker: 0
Loss: 19.18433952331543 for worker: 0
Loss: 18.02049446105957 for worker: 0
Loss: 16.92977523803711 for worker: 0
Loss: 15.907564163208008 for worker: 0
Loss: 14.949554443359375 for worker: 0
Loss: 14.0516996383667 for worker: 0
Loss: 13.210214614868164 for worker: 0
Loss: 12.421550750732422 for worker: 0
Loss: 11.682378768920898 for worker: 0
Loss: 10.989585876464844 for worker: 0
Loss: 10.340250015258789 for worker: 0
Loss: 9.731635093688965 for worker: 0
Loss: 9.161178588867188 for worker: 0
Loss: 8.62647819519043 for worker: 0
Loss: 8.125285148620605 for worker: 0
Loss: 7.655482769012451 for worker: 0
Loss: 7.21510124206543 for worker: 0


Linear(in_features=2, out_features=1, bias=True)